Step 1. Load necessary libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

Step 2. Acccess url and read html into list of dataframes. Index 0 have first table on that page

In [3]:
url = ("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# read url
Demo_data = pd.read_html(url, header=0)
Demo_data[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Step 3. Monipulating dataframe to do 
 a) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
#a
Demo_data[0].Neighborhood.fillna(Demo_data[0].Borough, inplace=True)
#b
Demo_data[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


 b) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
Demo_data[0]['Borough'].replace(['Not assigned'],np.nan,inplace=True)
Toronto_distr=Demo_data[0]
Toronto_distr.dropna(inplace=True)

c) More than one neighborhood can exist in one postal code area.These two rows will be combined into one row with the neighborhoods separated with a comma

In [6]:
Toronto_distr['Neighborhood']=Toronto_distr['Neighborhood'].str.replace('/',',',regex=True)
Toronto_distr.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

In [7]:
Toronto_distr.reset_index(drop=True, inplace=True)

In [8]:
Toronto_distr.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [9]:
Toronto_distr.shape

(103, 3)

Load coordinates from CSV file

In [10]:
csv_path = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
coord = pd.read_csv(csv_path) 
coord.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

In [11]:
coord.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge coordinates to Toronto districs dataframe

In [12]:
Toronto_distr_ = pd.merge(Toronto_distr, coord, on='Postalcode')
Toronto_distr_.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


Get York district (Toronto) coordinates to center map and add markers for all destricts  

In [65]:
address = 'York, Toronto, Canada'

geolocator = Nominatim(user_agent="sanych_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of York are 43.67910515, -79.49118414007154.


In [14]:
map_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_distr_['Latitude'], Toronto_distr_['Longitude'], Toronto_distr_['Borough'], Toronto_distr_['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

In [35]:
# The code was removed by Watson Studio for sharing.

Define function to access foursquare for venues in Toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        #print(LIMIT)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
Toronto_venues_ = getNearbyVenues(names=Toronto_distr_['Neighborhood'],
                                   latitudes=Toronto_distr_['Latitude'],
                                   longitudes=Toronto_distr_['Longitude']
                                  )

Parkwoods
200
Victoria Village
200
Regent Park , Harbourfront
200
Lawrence Manor , Lawrence Heights
200
Queen's Park , Ontario Provincial Government
200
Islington Avenue
200
Malvern , Rouge
200
Don Mills
200
Parkview Hill , Woodbine Gardens
200
Garden District , Ryerson
200
Glencairn
200
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
200
Rouge Hill , Port Union , Highland Creek
200
Don Mills
200
Woodbine Heights
200
St. James Town
200
Humewood-Cedarvale
200
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
200
Guildwood , Morningside , West Hill
200
The Beaches
200
Berczy Park
200
Caledonia-Fairbanks
200
Woburn
200
Leaside
200
Central Bay Street
200
Christie
200
Cedarbrae
200
Hillcrest Village
200
Bathurst Manor , Wilson Heights , Downsview North
200
Thorncliffe Park
200
Richmond , Adelaide , King
200
Dufferin , Dovercourt Village
200
Scarborough Village
200
Fairview , Henry Farm , Oriole
200
Northwood Park , York University
200
East Toronto
2

In [33]:
Toronto_venues_.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
4,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café


Calculate venues totals per neighborhood

In [34]:
Toronto_venues_.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,62,62,62,62,62,62
"Alderwood , Long Branch",48,48,48,48,48,48
"Bathurst Manor , Wilson Heights , Downsview North",40,40,40,40,40,40
Bayview Village,13,13,13,13,13,13
"Bedford Park , Lawrence Manor East",72,72,72,72,72,72
Berczy Park,100,100,100,100,100,100
"Birch Cliff , Cliffside West",13,13,13,13,13,13
"Brockton , Parkdale Village , Exhibition Place",100,100,100,100,100,100
Business reply mail Processing Centre,100,100,100,100,100,100


In [37]:
print('There are {} uniques categories.'.format(len(Toronto_venues_['Venue Category'].unique())))

There are 346 uniques categories.


Put venues types, normalize dataset and calculate frequency for each neighborhood

In [42]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues_[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues_['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculptu

In [45]:
Toronto_onehot.shape

(6917, 346)

In [47]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculptu

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 15 venues for each neighborhood for clustering later.

In [66]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Shopping Mall,Hong Kong Restaurant,Department Store,Caribbean Restaurant,Cantonese Restaurant,Bakery,Breakfast Spot,Gym / Fitness Center,Restaurant,Korean Restaurant,Supermarket,Food Court,Latin American Restaurant
1,"Alderwood , Long Branch",Park,Bank,Clothing Store,Toy / Game Store,Coffee Shop,Restaurant,Grocery Store,Café,Discount Store,Pizza Place,Pharmacy,Burger Joint,Electronics Store,Liquor Store,Pub
2,"Bathurst Manor , Wilson Heights , Downsview North",Park,Coffee Shop,Pizza Place,Gas Station,Bank,Trail,Sushi Restaurant,French Restaurant,Baseball Field,Middle Eastern Restaurant,Restaurant,Dog Run,Diner,Shopping Mall,Pharmacy
3,Bayview Village,Bank,Park,Trail,Gas Station,Café,Grocery Store,Chinese Restaurant,Japanese Restaurant,Restaurant,Farmers Market,Financial or Legal Service,Eastern European Restaurant,Egyptian Restaurant,Flea Market,Electronics Store
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Italian Restaurant,Bakery,Sushi Restaurant,Bagel Shop,Café,Pub,Asian Restaurant,Pizza Place,Fast Food Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Sports Club,Gastropub


Clustering neighborhoods into 5 clusters using k-means method.

In [68]:
# set number of clusters
kclusters = 5

Toronto_grouped_clust = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clust)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 0, 0, 2, 1, 0, 1, 1, 1], dtype=int32)

In [69]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


Add cluster nomber to top 15 dataframe to see cluster details later

In [71]:

Toronto_merged = Toronto_distr_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Pharmacy,Coffee Shop,Supermarket,Fast Food Restaurant,Bus Stop,Bank,Gas Station,Sandwich Place,BBQ Joint,Office,Caribbean Restaurant,Pizza Place,Beer Store,Intersection,Liquor Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Gym,Middle Eastern Restaurant,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Hockey Arena,Beer Store,Food,Cosmetics Shop,Soccer Field,Bus Line,Shopping Mall,Thai Restaurant,Shoe Store
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Café,Diner,Restaurant,Park,Japanese Restaurant,Pub,Italian Restaurant,Thai Restaurant,Food Truck,Farmers Market,Middle Eastern Restaurant,Athletics & Sports,Performing Arts Venue,Gastropub
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Dessert Shop,Sandwich Place,Furniture / Home Store,Vietnamese Restaurant,Fried Chicken Joint,Toy / Game Store,Bookstore,Latin American Restaurant,Food Court,Turkish Restaurant,Greek Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Ramen Restaurant,Gastropub,Hotel,Park,Breakfast Spot,Thai Restaurant,Dance Studio,Plaza,Comic Shop,Arts & Crafts Store,Supermarket


Visualize clusters on the map

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Here are details of each cluster

In [73]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
5,Etobicoke,0,Park,Pharmacy,Shopping Mall,Bank,Playground,Camera Store,Café,Golf Course,Liquor Store,Grocery Store,Bus Stop,Skating Rink,Supermarket,Bus Line,Spa
11,Etobicoke,0,Park,Hotel,Pizza Place,Bank,Breakfast Spot,Sandwich Place,Restaurant,Clothing Store,Coffee Shop,Convenience Store,Pharmacy,Theater,Electronics Store,Farmers Market,Fish & Chips Shop
12,Scarborough,0,Park,Hotel,Italian Restaurant,Burger Joint,Grocery Store,Gym,Gym / Fitness Center,Breakfast Spot,Department Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
28,North York,0,Park,Coffee Shop,Pizza Place,Gas Station,Bank,Trail,Sushi Restaurant,French Restaurant,Baseball Field,Middle Eastern Restaurant,Restaurant,Dog Run,Diner,Shopping Mall,Pharmacy
39,North York,0,Bank,Park,Trail,Gas Station,Café,Grocery Store,Chinese Restaurant,Japanese Restaurant,Restaurant,Farmers Market,Financial or Legal Service,Eastern European Restaurant,Egyptian Restaurant,Flea Market,Electronics Store
50,North York,0,Park,Electronics Store,Asian Restaurant,Skating Rink,Mexican Restaurant,Bakery,Latin American Restaurant,Café,Italian Restaurant,Shopping Mall,General Entertainment,Bank,Pharmacy,Vietnamese Restaurant,Sports Bar
51,Scarborough,0,Park,Harbor / Marina,Pizza Place,Grocery Store,Fast Food Restaurant,Beach,Pub,Sandwich Place,Pharmacy,Discount Store,Ice Cream Shop,Coffee Shop,Pet Store,Cajun / Creole Restaurant,Sushi Restaurant
58,Scarborough,0,Park,Hotel,Restaurant,Filipino Restaurant,Café,General Entertainment,Thai Restaurant,Gym,Diner,Golf Course,College Stadium,Ice Cream Shop,Deli / Bodega,Farm,Creperie
88,Etobicoke,0,Park,Bakery,Café,Indian Restaurant,Grocery Store,Pharmacy,Beer Store,Thai Restaurant,Mexican Restaurant,Bar,Restaurant,Supermarket,Sushi Restaurant,Italian Restaurant,Fried Chicken Joint


In [74]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Café,Diner,Restaurant,Park,Japanese Restaurant,Pub,Italian Restaurant,Thai Restaurant,Food Truck,Farmers Market,Middle Eastern Restaurant,Athletics & Sports,Performing Arts Venue,Gastropub
4,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Ramen Restaurant,Gastropub,Hotel,Park,Breakfast Spot,Thai Restaurant,Dance Studio,Plaza,Comic Shop,Arts & Crafts Store,Supermarket
9,Downtown Toronto,1,Coffee Shop,Café,Gastropub,Japanese Restaurant,Restaurant,Theater,Italian Restaurant,Hotel,Seafood Restaurant,Ramen Restaurant,Plaza,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Diner
15,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Thai Restaurant,Italian Restaurant,Bakery,Plaza,Farmers Market,Seafood Restaurant,American Restaurant,Sushi Restaurant,Cosmetics Shop,Japanese Restaurant,Theater
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Plaza,Concert Hall,Bookstore,Thai Restaurant,Park,Cosmetics Shop,Bakery,Dessert Shop,Japanese Restaurant,Theater
24,Downtown Toronto,1,Coffee Shop,Park,Café,Pizza Place,Restaurant,Italian Restaurant,Sushi Restaurant,Japanese Restaurant,Yoga Studio,Plaza,Dance Studio,Thai Restaurant,Theater,Smoke Shop,Diner
25,Downtown Toronto,1,Café,Coffee Shop,Indian Restaurant,Ice Cream Shop,Italian Restaurant,Grocery Store,Dessert Shop,Bar,Bakery,Cocktail Bar,Korean Restaurant,South American Restaurant,Park,Japanese Restaurant,Restaurant
30,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Park,Plaza,Sandwich Place,Vegetarian / Vegan Restaurant,Hotel,Yoga Studio,Italian Restaurant,Pizza Place,Thai Restaurant,Theater,Sporting Goods Shop,Cosmetics Shop
31,West Toronto,1,Café,Italian Restaurant,Coffee Shop,Bar,Bakery,Sushi Restaurant,Park,Thai Restaurant,Gastropub,Mexican Restaurant,Sandwich Place,Diner,Brewery,Breakfast Spot,Brazilian Restaurant
35,East York,1,Greek Restaurant,Café,Coffee Shop,Pizza Place,Thai Restaurant,Bakery,Ice Cream Shop,Ethiopian Restaurant,Park,Gastropub,American Restaurant,Cosmetics Shop,Brewery,Beer Bar,Pub


In [75]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,North York,2,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Dessert Shop,Sandwich Place,Furniture / Home Store,Vietnamese Restaurant,Fried Chicken Joint,Toy / Game Store,Bookstore,Latin American Restaurant,Food Court,Turkish Restaurant,Greek Restaurant
6,Scarborough,2,Zoo Exhibit,Fast Food Restaurant,Restaurant,Pizza Place,Coffee Shop,Big Box Store,Spa,Bus Station,Business Service,Caribbean Restaurant,Chinese Restaurant,Cosmetics Shop,Bank,Paper / Office Supplies Store,Other Great Outdoors
7,North York,2,Coffee Shop,Japanese Restaurant,Restaurant,Bank,Pizza Place,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Park,Gym,Asian Restaurant,Pharmacy,Supermarket,Burger Joint,Liquor Store
13,North York,2,Coffee Shop,Japanese Restaurant,Restaurant,Bank,Pizza Place,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Park,Gym,Asian Restaurant,Pharmacy,Supermarket,Burger Joint,Liquor Store
16,York,2,Coffee Shop,Café,Italian Restaurant,Bank,Grocery Store,Pizza Place,Sandwich Place,Caribbean Restaurant,Indian Restaurant,Burger Joint,Restaurant,BBQ Joint,Trail,Frozen Yogurt Shop,Bagel Shop
19,East Toronto,2,Coffee Shop,Pub,Grocery Store,Japanese Restaurant,Breakfast Spot,Bakery,Sandwich Place,BBQ Joint,Ice Cream Shop,Beach,Mexican Restaurant,Tea Room,Gas Station,Pizza Place,Middle Eastern Restaurant
21,York,2,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bakery,Grocery Store,Café,Pizza Place,Convenience Store,Sushi Restaurant,Bank,Caribbean Restaurant,Sandwich Place,Park,Mexican Restaurant,Pharmacy
23,East York,2,Coffee Shop,Indian Restaurant,Bakery,Grocery Store,Restaurant,Pizza Place,Electronics Store,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Sandwich Place,Park,Gym,Pet Store
26,Scarborough,2,Coffee Shop,Indian Restaurant,Gas Station,Restaurant,Sandwich Place,Clothing Store,Light Rail Station,Pharmacy,Bakery,Bank,Sporting Goods Shop,Hotel,Thai Restaurant,Italian Restaurant,Bar
29,East York,2,Coffee Shop,Sandwich Place,Restaurant,Bank,Electronics Store,Burger Joint,Sporting Goods Shop,Grocery Store,Furniture / Home Store,Indian Restaurant,Turkish Restaurant,Japanese Restaurant,Supermarket,Beer Store,Bakery


In [76]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,North York,3,Pharmacy,Coffee Shop,Supermarket,Fast Food Restaurant,Bus Stop,Bank,Gas Station,Sandwich Place,BBQ Joint,Office,Caribbean Restaurant,Pizza Place,Beer Store,Intersection,Liquor Store
1,North York,3,Coffee Shop,Gym,Middle Eastern Restaurant,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Hockey Arena,Beer Store,Food,Cosmetics Shop,Soccer Field,Bus Line,Shopping Mall,Thai Restaurant,Shoe Store
8,East York,3,Pizza Place,Pharmacy,Park,Fast Food Restaurant,Convenience Store,Intersection,Gym / Fitness Center,Sandwich Place,Gastropub,Men's Store,Mexican Restaurant,Soccer Stadium,Skating Rink,Rock Climbing Spot,Bakery
10,North York,3,Coffee Shop,Fast Food Restaurant,Bank,Grocery Store,Restaurant,Park,Gas Station,Fried Chicken Joint,Sandwich Place,Caribbean Restaurant,Playground,Pizza Place,Bowling Alley,Dessert Shop,Seafood Restaurant
14,East York,3,Coffee Shop,Park,Pizza Place,Pharmacy,Thai Restaurant,Sandwich Place,Bar,Bank,Gastropub,Pet Store,Grocery Store,Ice Cream Shop,Cosmetics Shop,Arts & Crafts Store,Pub
17,Etobicoke,3,Baseball Field,Coffee Shop,Bank,Grocery Store,Convenience Store,Café,Pizza Place,Garden,Sandwich Place,Gas Station,Donut Shop,Restaurant,Discount Store,Recreation Center,Liquor Store
18,Scarborough,3,Pizza Place,Breakfast Spot,Fast Food Restaurant,Bank,Grocery Store,Juice Bar,Coffee Shop,Park,Discount Store,Restaurant,Sandwich Place,Clothing Store,Automotive Shop,Sports Bar,Fried Chicken Joint
22,Scarborough,3,Coffee Shop,Pharmacy,Fast Food Restaurant,Bank,Pizza Place,Park,Indian Restaurant,Juice Bar,Gym,Café,Music Store,Thrift / Vintage Store,Chinese Restaurant,Beer Store,Discount Store
27,North York,3,Coffee Shop,Chinese Restaurant,Pharmacy,Bank,Sandwich Place,Park,Bakery,Sushi Restaurant,Pizza Place,Grocery Store,Shopping Mall,Supermarket,Campground,Intersection,Szechuan Restaurant
32,Scarborough,3,Sandwich Place,Pharmacy,Bank,Ice Cream Shop,Pizza Place,Coffee Shop,Convenience Store,Fast Food Restaurant,Gas Station,Xinjiang Restaurant,Liquor Store,Chinese Restaurant,Beer Store,Big Box Store,Grocery Store


In [77]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
95,Scarborough,4,Farm,National Park,Zoo,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field,Dumpling Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
